In [1]:
### DATA TREATMENT SECTION ###

from sys import exit

from astropy.io.votable import is_votable, parse
from astropy.table import Table
from astropy import units as u
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
from matplotlib.markers import MarkerStyle
# from scipy.optimize import curve_fit
# from scipy.stats.mstats import chisquare

In [3]:
def is_VOtable(fullname):
    """
    Check whether a file is a VOtable.
    
    Input
    ------
    fullname : string
        path+name of the file to test
    
    Returns True if it is a VOtable. False otherwise.
    """
    tag = is_votable(fullname)
    print("The file", fullname, "is a VOtable, right ?", tag)
    return tag

def linear_fit(x, A, offset):
    """
    Compute a linear relation A*x+offset.
    
    Input
    -----
    x : numpy array
        input data
    A : float
        Slope coefficient
    offset : float
        x=0 Y-coordinate
        
    Returns a numpy array A*x+offset.
    """
    return A*x+offset

def maskToRemoveVal(listOfArrays, val=None, keep=True):
    """
    Computes a mask by finding occurences in a list of arrays.
    
    Input
    -----
    listOfArrays : list of numpy arrays
        the list of arrays from which the mask is built
    val : float or None
        the value to find. If val=None, it looks for nan values.
    keep : boolean
        if True, it builds a mask with True everywhere the value val is encountered. If False, it does the opposite
    
    Returns a mask as a numpy array.
    """
    
    shp    = listOfArrays[0].shape
    #Checking that arrays have the same shape
    for array in listOfArrays[1:]:
        if shp != array.shape:
            exit("Arrays do not have the same dimensions, thus making the masking operation not fit. Exiting.")
  
    #Constructing first mask
    if val is None:
        tmp    = np.logical_not(np.isnan(listOfArrays[0]))
    else:
        tmp = listOfArrays[0] == val
        if not keep:
            tmp = np.logical_not(tmp)
        
    #Applying logical and on all the masks
    for (num, array) in enumerate(listOfArrays[1:]):
        #consider we are looking for nan in the arrays
        if val is None:
            tmp = np.logical_and(tmp, np.logical_not(np.isnan(array)))
        else:
            if keep:
                tmp = np.logical_and(tmp, array==val)
            else:
                tmp = np.logical_and(tmp, array != val)
    return tmp

def applyMask(listOfArrays, mask):
    """
    Apply the same mask to a list of arrays and return the new arrays.
    
    Input
    -----
    listOfArrays : list of numpy arrays
        the list of arrays the mask is applied to
    mask : numpy array
        the mask to apply
        
    Returns the reduced list of arrays
    """
    
    for (num, array) in enumerate(listOfArrays):
        listOfArrays[num] = array[mask]
    return listOfArrays

def findWhereIsValue(listOfArrays, val=None):
    """
    Find and print the first position where a value is found within a list of arrays.
    
    Input
    -----
    listOfArrays : list of numpy arrays
        list from which the value val is searched
    val : float or None
        value to look for. If val=None, it looks for nan values.
    """
    
    for (num, array) in enumerate(listOfArrays):
        if val is None:
            if np.any(np.isnan(array)):
                print("A nan was found at position", np.where(np.isnan(array))[0], "within array number", num)
            else:
                print("No nan was found in array number", num)
        else:
            if np.asarray(np.where(array==val)).shape[1] == 0:
                print("No value", val, "found within array number", num)
            else:
                print("Value", val, "found at position", np.where((array==val))[0], "within array number", num)
                
                
def asManyPlots(numPlot, datax, datay, hideXlabel=False, hideYlabel=False, hideYticks=False,
                placeYaxisOnRight=False, xlabel="", ylabel='', marker='o', color='black', plotFlag=True,
                label='', zorder=0, textsize=24, showLegend=False, legendTextSize=24, linestyle='None',
                ylim=[None, None], xlim=[None, None], cmap=None, cmapMin=None, cmapMax=None,
                showColorbar=False, locLegend='best', tickSize=24):
    """
    Function which plots on a configurable subplot grid either with pyplot.plot or pyplot.scatter.
    
    Input
    -----
    numPlot : int (3 digits)
        the subplot number
    datax: list, numpy array
        the x data
    datay :list, numpy array
        the y data
    hideXlabel : boolean
        whether to hide the x label or not
    hideYlabel : boolean
        whether to hide the y label or not
    hideYticks : boolean
        whether to hide the y ticks or not
    placeYaxisOnRight : boolean
        whether to place the y axis of the plot on the right or not
    xlabel : string
        the x label
    ylabel : string
        the y label
    marker : string, char (pyplot symbol)
        the marker to use for the data
    color : string/char/RGB/list of values
        the color for the data. If a list of values is given, plotFlag must be False and a cmap must be given.
    plotFlag : boolean
        if True, plots with pyplot.plot function. If False, use pyplot.scatter
    label : string
        legend label for the data
    zorder : int
        whether the data will be plot in first position or in last. The lower the value, the earlier it will be plotted
    textsize : int
        size for the labels
    showLegend : boolean
        whether to show the legend or not
    legendTextSize : int
        size for the legend
    linestyle : string
        which line style to use
    ylim : list of floats/None
        the y-axis limits to use. If None is specified as lower/upper/both limit(s), the minimum/maximum/both values are used
    xlim : list of floats/None
        the x-axis limits to use. If None is specified as lower/upper/both limit(s), the minimum/maximum/both values are used
    cmap : matplotlib colormap
        the colormap to use for the scatter plot only
    cmapMin: float
        the minmum value for the colormap
    cmapMax: float
        the maximum value for the colormap
    showColorbar : boolean
        whether to show the colorbar for a scatter plot or not
    locLegend : string, int
        position where to place the legend
    tickSize : int
        size of the ticks on both axes
        
    Return current axis and last plot.
    """
    
    ax1 = plt.subplot(numPlot)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.tick_params(which='both', direction='in', labelsize=tickSize)
    plt.grid()
    
    #hiding labels if required
    if hideXlabel:
        ax1.axes.get_xaxis().set_ticklabels([])
    else:
        plt.xlabel(xlabel, size=textsize)    
    if hideYticks:
        ax1.axes.get_yaxis().set_ticklabels([])
    if not hideYlabel:    
        plt.ylabel(ylabel, size=textsize)
    
    #Place Y axis on the right if required
    if placeYaxisOnRight:
        ax1.yaxis.tick_right()
        ax1.yaxis.set_label_position("right")
    
    #Plotting
    if plotFlag:
        tmp = plt.plot(datax, datay, label=label, marker=marker, color=color, zorder=zorder, linestyle=linestyle)
    else:
        if cmapMin is None:
            cmapMin = np.min(color)
        if cmapMax is None:
            cmapMax = np.max(color)
        
        tmp = plt.scatter(datax, datay, label=label, marker=marker, c=color, zorder=zorder, 
                          cmap=cmap, vmin=cmapMin, vmax=cmapMax)
        
        if showColorbar:
            plt.colorbar(tmp)
            
    if showLegend:
        plt.legend(loc=locLegend, prop={'size': legendTextSize})
        
    #Define Y limits if required
    if ylim[0] is not None:
        ax1.set_ylim(bottom=ylim[0])
#     else:
#         ax1.set_ylim(bottom=ax.get_ylim()[0])
    if ylim[1] is not None:
        ax1.set_ylim(top=ylim[1])
#     else:
#         ax1.set_ylim(top=ax.get_ylim()[1])
        
    #define X limits if required
    if xlim[0] is not None:
        ax1.set_xlim(left=xlim[0])
#     else:
#         ax1.set_xlim(left=ax.get_xlim()[0])
    if xlim[1] is not None:
        ax1.set_xlim(right=xlim[1])
#     else:
#         ax1.set_xlim(right=ax.get_xlim()[1])
    
    return ax1, tmp

In [4]:
pathdata = "outputs/"
data     = ["matching_fieldGals_with_Zurich_acc_1_arcsec.vot", "master_field_galaxiesV2.vot"]

#Checking that file format is correct
for name in data:
    voTag = is_VOtable(pathdata+name)
    if voTag:        
        fullFileName = pathdata + name
        #Retrieving the data
        table = parse(fullFileName)
        full  = table.get_first_table()
        
        print("Size of", name, "is", full.array.shape[0], "\n")
    else:
        exit("Exiting")

The file outputs/matching_fieldGals_with_Zurich_acc_1_arcsec.vot is a VOtable, right ? True


Size of matching_fieldGals_with_Zurich_acc_1_arcsec.vot is 190 

The file outputs/master_field_galaxiesV2.vot is a VOtable, right ? True
Size of master_field_galaxiesV2.vot is 545 



In [5]:
#Getting data
matchWithZurich = parse(pathdata+data[0]).get_first_table().array
fieldGals       = parse(pathdata+data[1]).get_first_table().array

In [25]:
#Checking that the matching procedure did not duplicate galaxies
master = [matchWithZurich, fieldGals]

for catalog, nameCat in zip(master, data):
    cnt = True
    for ra, dec, nb in zip(catalog['RA'], catalog['DEC'], range(catalog['RA'].shape[0])):
        
        where1 = np.where(catalog['RA']==ra)[0]
        where2 = np.where(catalog['DEC']==dec)[0]
        
        if (len(where1)>1) and (len(where2)>1):
            
            flag = True
            for w in where2:
                
                if flag and (w in where1):
                    print("RA =", ra, "deg and DEC =", dec, "deg galaxy (line " + str(nb) + ") is present more than once in catalog", nameCat)
                    flag = False
                    cnt  = False
    if cnt:
        print("All the galaxies are only listed once in the catalog", nameCat)

All the galaxies are only listed once in the catalog matching_fieldGals_with_Zurich_acc_1_arcsec.vot
RA = 150.053047 deg and DEC = 2.604346 deg galaxy (line 6) is present more than once in catalog master_field_galaxiesV2.vot
RA = 150.053047 deg and DEC = 2.604346 deg galaxy (line 70) is present more than once in catalog master_field_galaxiesV2.vot


In [28]:
#Converting to an astropy table for simplicity
tableMatch = Table(matchWithZurich)        
tableField = Table(fieldGals)

In [30]:
#Checking that the maximum angular separation is less than 1 arcsec in the matching tableMatch
print("Maximum separation is", str((tableMatch['Separation']*u.arcsec).max()) + ".")
print("Mean separation is", str(np.mean(tableMatch['Separation']*u.arcsec)) + ".")
print("Median separation is", str(np.median(tableMatch['Separation']*u.arcsec)) + ".")
print("1st quantile is", str(np.quantile(tableMatch['Separation'], 0.25)*u.arcsec) + ".")
print("3rd quantile is", str(np.quantile(tableMatch['Separation'], 0.75)*u.arcsec) + ".")

Maximum separation is 0.8778970454248154 arcsec.
Mean separation is 0.10395451693170249 arcsec.
Median separation is 0.06814922333003948 arcsec.
1st quantile is 0.04285471659041162 arcsec.
3rd quantile is 0.12246327531669089 arcsec.


In [31]:
tableField.dtype.names

('ID_Laigle_16',
 'RA',
 'DEC',
 'Z_MUSE',
 'CONFID',
 'Blend',
 'Defect',
 'Revisit',
 'ALPHA_J2000',
 'DELTA_J2000',
 'NUMBER',
 'X_IMAGE',
 'Y_IMAGE',
 'ERRX2_IMAGE',
 'ERRY2_IMAGE',
 'ERRXY_IMAGE',
 'FLAG_HJMCC',
 'FLUX_RADIUS',
 'KRON_RADIUS',
 'EBV',
 'FLAG_PETER',
 'FLAG_COSMOS',
 'FLAG_DEEP',
 'FLAG_SHALLOW',
 'Ks_FLUX_APER2',
 'Ks_FLUXERR_APER2',
 'Ks_FLUX_APER3',
 'Ks_FLUXERR_APER3',
 'Ks_MAG_APER2',
 'Ks_MAGERR_APER2',
 'Ks_MAG_APER3',
 'Ks_MAGERR_APER3',
 'Ks_MAG_AUTO',
 'Ks_MAGERR_AUTO',
 'Ks_MAG_ISO',
 'Ks_MAGERR_ISO',
 'Ks_FLAGS',
 'Ks_IMAFLAGS_ISO',
 'Y_FLUX_APER2',
 'Y_FLUXERR_APER2',
 'Y_FLUX_APER3',
 'Y_FLUXERR_APER3',
 'Y_MAG_APER2',
 'Y_MAGERR_APER2',
 'Y_MAG_APER3',
 'Y_MAGERR_APER3',
 'Y_MAG_AUTO',
 'Y_MAGERR_AUTO',
 'Y_MAG_ISO',
 'Y_MAGERR_ISO',
 'Y_FLAGS',
 'Y_IMAFLAGS_ISO',
 'H_FLUX_APER2',
 'H_FLUXERR_APER2',
 'H_FLUX_APER3',
 'H_FLUXERR_APER3',
 'H_MAG_APER2',
 'H_MAGERR_APER2',
 'H_MAG_APER3',
 'H_MAGERR_APER3',
 'H_MAG_AUTO',
 'H_MAGERR_AUTO',
 'H_MAG_ISO'

In [34]:
#Remove galaxies which have wrong ID_Laigle_16 values (9999)
tableMatch = tableMatch[tableMatch != 9999]
tableField = tableField[tableField != 9999]

findWhereIsValue([tableMatch, tableField], 9999)

/home/wilfried/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:2643: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = (self.as_array().data == other) & (self.mask == false_mask)


No value 9999 found within array number 0
No value 9999 found within array number 1
